# 고도몰 SMS 발송내역 분석

8~11월 월별 SMS 발신내역 통계 확인
- 고도몰어드민 SMS 발송 내역보기: http://gdadmin.planbio.godomall.com/member/sms_log.php
- 고도몰 DB 정보: http://doc.godomall5.godomall.com/godo/database/table_layout.php
  - es_smsSendLog, es_smsSendList 테이블 확인


In [1]:
import json
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots

df_raw_log = pd.read_excel("data/es_smsLog_20241120.xlsx")
df_raw_list = pd.read_excel("data/es_smsSendList_20241120.xlsx")

## 분석 데이터 만들기

### smsLog 데이터 클랜징

In [2]:
log_columns = [
    "sno",
    "sendFl",
    "smsType",
    "sender",
    "contents",
    "receiverCnt",
    "receiverType",
    "receiverInfo",
    "sendStatus",
    "sendSuccessCnt",
    "sendFailCnt",
    "sendDt",
    "smsSendKey",
]
df_log = df_raw_log.copy()[log_columns]

# 카카오 알림톡 필터링
df_log = df_log[df_log["sendFl"] != "kakao"]

# Convert columns to appropriate data types
df_log["sno"] = df_log["sno"].astype(int)
df_log["sendFl"] = df_log["sendFl"].astype("category")
df_log["smsType"] = df_log["smsType"].astype("category")
df_log["sender"] = df_log["sender"].astype(str)
df_log["contents"] = df_log["contents"].astype(str)
df_log["receiverCnt"] = df_log["receiverCnt"].astype(int)
df_log["receiverType"] = df_log["receiverType"].astype("category")
df_log["sendStatus"] = df_log["sendStatus"].astype("category")
df_log["sendSuccessCnt"] = df_log["sendSuccessCnt"].astype(int)
df_log["sendFailCnt"] = df_log["sendFailCnt"].astype(int)
df_log["sendDt"] = pd.to_datetime(df_log["sendDt"], errors="coerce")
df_log["smsSendKey"] = df_log["smsSendKey"].astype(int)


# sender 데이터 다듬기
def extract_sender(string):
    arr = json.loads(string.replace("\\", ""))
    return arr[1]


# receiverInfo 데이터 다듬기
def extract_content_type(string):
    try:
        data = json.loads(string)
    except:
        # print(string)
        return "FAILED"

    try:
        return data["smsAutoCode"]
    except:
        # print(string)
        return "ETC"


df_log["sender"] = df_log["sender"].apply(extract_sender)
df_log["contentType"] = df_log["receiverInfo"].apply(extract_content_type)

condition_incash = (df_log.contentType == "FAILED") & (
    df_log.contents.str.contains("결제완료")
)
df_log.loc[condition_incash, "contentType"] = "INCASH"

condition_repay = (df_log.contentType == "FAILED") & (
    df_log.contents.str.contains("환불")
)
df_log.loc[condition_repay, "contentType"] = "REPAY"

condition_order = (df_log.contentType == "FAILED") & (
    df_log.contents.str.contains("접수")
)
df_log.loc[condition_order, "contentType"] = "ORDER"

### smsList 데이터 클랜징

In [3]:
list_columns = [
    "sno",
    "smsLogSno",
    "receiverName",
    "receiverCellPhone",
    "sendCheckFl",
    "acceptCheckFl",
    "regDt",
    "kakaoSendKey",
]
df_list = df_raw_list.copy()[list_columns]
# Convert columns to appropriate data types
df_list["sno"] = df_list["sno"].astype(int)
df_list["smsLogSno"] = df_list["smsLogSno"].astype(int)
df_list["receiverName"] = df_list["receiverName"].astype(str)
df_list["receiverCellPhone"] = df_list["receiverCellPhone"].astype(str)
df_list["sendCheckFl"] = df_list["sendCheckFl"].astype("category")
df_list["acceptCheckFl"] = df_list["acceptCheckFl"].astype("category")
df_list["regDt"] = pd.to_datetime(df_list["regDt"], errors="coerce")
df_list["kakaoSendKey"] = df_list["kakaoSendKey"].astype(float)

# 카카오 알림톡 필터링
df_list = df_list[df_list.kakaoSendKey.isna()]
df_list = df_list.drop("kakaoSendKey", axis=1)

### 최종 분석 데이터
- list 데이터에 log의 `["sno", "sendFl", "smsType", "sender", "receiverType", "contentType"]` 컬럼데이터 추가

In [4]:
# Merge df_list with df_log to add sendFl, smsType, sender, receiverType
df = df_list.merge(
    df_log[["sno", "sendFl", "smsType", "sender", "receiverType", "contentType"]],
    left_on="smsLogSno",
    right_on="sno",
    how="left",
)

df = df.drop(columns=["sno_y"])
df = df.rename(columns={"sno_x": "sno"})
df["month"] = df["regDt"].dt.month
df = df.dropna()

In [5]:
def get_content_type_kr(content_type):
    content_type_kr = {
        "INCASH": "결제완료",
        "DEPOSIT_MINUS": "페이차감",
        "ORDER": "주문접수",
        "DEPOSIT_PLUS": "페이지급",
        "JOIN": "가입신청",
        "SOLD_OUT": "품절",
        "SLEEP_INFO_TODAY": "휴면회원전환",
        "ETC": "기타",
        "GROUP_CHANGE": "회원등급변경",
        "REPAY": "환불완료",
        "REFUND": "환불신청",
        "SLEEP_INFO": "휴면회원전환예정",
        "APPROVAL": "가입승인",
        "CANCEL": "주문취소",
        "PASS_AUTH": "회원인증번호",
        "ADMIN_APPROVAL": "교환/환불승인",
        "PLANDOCS": "제품요청 게시판",
        "qa": "1:1문의 게시판",
        "goodsqa": "상품문의 게시판",
        "AGREEMENT2YPERIOD": "광고수신동의안내",
        "sample": "샘플요청 게시판",
        "EXCHANGE": "교환신청",
        "BACK": "반품신청",
        "salesman": "영업사원방문 게시판",
        "INVOICE_CODE": "송장번호",
    }
    try:
        return content_type_kr[content_type]
    except:
        return "기타"


df["contentTypeKr"] = df["contentType"].apply(get_content_type_kr)

df_user = df[df["receiverType"] == "each"]
df_admin = df[df["receiverType"] == "group"]

In [6]:
def draw_plot_by_sms_type(df, title):
    plot_data = df.contentTypeKr.value_counts()

    fig = px.bar(
        plot_data,
        x=plot_data.index,
        y=plot_data.values,
        title=title,
        labels={"contentTypeKr": "", "y": "발송수"},
    )
    fig.show()

In [7]:
def draw_month_counts_by_sms_type(df, title):
    df_08 = df[df["month"] == 8]["contentTypeKr"].value_counts()
    df_09 = df[df["month"] == 9]["contentTypeKr"].value_counts()
    df_10 = df[df["month"] == 10]["contentTypeKr"].value_counts()
    df_11 = df[df["month"] == 11]["contentTypeKr"].value_counts()

    df_combined = pd.concat(
        [df_11, df_10, df_09, df_08],
        axis=1,
        keys=["11월", "10월", "9월", "8월"],
    )

    df_combined = df_combined.fillna(0)
    df_combined = df_combined.astype(int)
    df_combined = df_combined.sort_values(by="11월", ascending=False)

    display(df_combined)

    fig = make_subplots(
        rows=1,
        cols=4,
        specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}]],
        subplot_titles=["11월", "10월", "9월", "8월"],
    )

    fig.add_trace(
        px.pie(df_11, values=df_11.values, names=df_11.index).data[0],
        row=1,
        col=1,
    )
    fig.add_trace(
        px.pie(df_10, values=df_10.values, names=df_10.index).data[0],
        row=1,
        col=2,
    )
    fig.add_trace(
        px.pie(df_09, values=df_09.values, names=df_09.index).data[0],
        row=1,
        col=3,
    )
    fig.add_trace(
        px.pie(df_08, values=df_08.values, names=df_08.index).data[0],
        row=1,
        col=4,
    )

    fig.update_layout(title_text=title)
    fig.show()

In [8]:
def draw_month_counts(category, df, title="", barmode="stack"):
    # Group by month and category, then count occurrences
    month_counts = (
        df.groupby(["month", category], observed=True).size().unstack(fill_value=0)
    )
    month_counts = month_counts.sort_index(ascending=False)

    # Map month numbers to month names
    month_counts.index = month_counts.index.map(
        {
            11: "11월",
            10: "10월",
            9: "9월",
            8: "8월",
        }
    )
    display(month_counts.T)

    # Plot the data using plotly
    fig = px.bar(
        month_counts,
        x=month_counts.index,
        y=month_counts.columns,
        title=title,
        labels={"value": "", "month": ""},
        barmode=barmode,
    )

    fig.show()

## 회원/운영자 월별 전송 통계

In [9]:
draw_month_counts("receiverType", df, "회원/운영자 월별 전송 통계")

month,11월,10월,9월,8월
receiverType,,,,
each,2193,2138,2240,1864
group,14240,21467,21343,20054


## 문자별 발송 건 수 분석

In [10]:
draw_plot_by_sms_type(df_user, "회원에게 보내는 문자별 발송 건수 (8.1 ~ 11.20)")

In [11]:
draw_plot_by_sms_type(df_admin, "운영자에게 보내는 문자별 발송 건수 (8.1 ~ 11.20)")

In [12]:
draw_month_counts_by_sms_type(
    df_user,
    "회원에게 보내는 문자별 발송 건수 (월별 비중)",
)

,11월,10월,9월,8월
contentTypeKr,,,,
페이차감,820,1098,1079,1101
결제완료,557,0,0,0
페이지급,218,308,309,261
주문접수,153,154,191,137
환불완료,102,89,61,51
회원등급변경,73,97,72,51
가입신청,44,87,59,86
가입승인,35,70,45,35
교환/환불승인,34,11,28,16


In [13]:
draw_month_counts_by_sms_type(
    df_admin,
    "운영자에게 보내는 문자별 발신 건수 (월별 비중)",
)

,11월,10월,9월,8월
contentTypeKr,,,,
결제완료,10844,17300,16700,16702
주문접수,1225,1530,1890,1370
품절,1032,1190,1290,640
환불신청,473,430,590,460
가입신청,283,602,413,602
주문취소,208,140,260,30
반품신청,64,60,30,20
1:1문의 게시판,27,40,50,40
상품문의 게시판,22,35,25,25


## SMS/LMS 별 월별 전송 통계

In [14]:
draw_month_counts("sendFl", df, "SMS/LMS 별 월별 전송 통계 (전체)", "group")

month,11월,10월,9월,8월
sendFl,,,,
lms,3245,3274,3494,2448
sms,13188,20331,20089,19470


In [15]:
draw_month_counts("sendFl", df_user, "SMS/LMS 별 월별 전송 통계 (회원)", "group")

month,11월,10월,9월,8월
sendFl,,,,
lms,2123,2074,2164,1798
sms,70,64,76,66


In [16]:
draw_month_counts("sendFl", df_admin, "SMS/LMS 별 월별 전송 통계 (운영자)", "group")

month,11월,10월,9월,8월
sendFl,,,,
lms,1122,1200,1330,650
sms,13118,20267,20013,19404


## 문자카테고리 별 월별 전송 통계

In [17]:
draw_month_counts("smsType", df, "문자카테고리 별 월별 전송 통계 (전체)", "group")

month,11월,10월,9월,8월
smsType,,,,
board,114,203,152,184
member,1563,2419,2299,2208
order,14739,20958,21106,19500
user,17,25,26,26


In [18]:
draw_month_counts("smsType", df_user, "문자카테고리 별 월별 전송 통계 (회원)", "group")

month,11월,10월,9월,8월
smsType,,,,
board,19,28,22,24
member,1280,1817,1886,1606
order,877,268,306,208
user,17,25,26,26


In [19]:
draw_month_counts(
    "smsType", df_admin, "문자카테고리 별 월별 전송 통계 (운영자)", "group"
)

month,11월,10월,9월,8월
smsType,,,,
board,95,175,130,160
member,283,602,413,602
order,13862,20690,20800,19292
